In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler 

In [14]:
xl = pd.ExcelFile("sarp.xlsx")
ds = xl.parse('sarp')
X_uncleaned = ds.loc[:, "Baseline_preDrug_FEV1pp":"Baseline_preDrug_FEV1_FVC"]
imputer = Imputer(strategy="median")
imputer.fit(X_uncleaned)
X_cleaned = imputer.transform(X_uncleaned)
Xds = pd.DataFrame(X_cleaned, columns=X_uncleaned.columns).astype(np.float32)
age = ds.loc[:,"ageasthonset"].astype(np.float32)
Xds = Xds["Baseline_preDrug_FEV1_FVC"]
Xds = pd.DataFrame(Xds)
Xds = Xds.join(age)
Xds = Xds.join(ds["gender"])
#Xpd = Xds
Xds = pd.DataFrame.as_matrix(Xds)
yds = ds['status_factorized'].astype(np.int32)
#yds = pd.DataFrame.as_matrix(yds)
yds = yds.values.ravel()
#ypd = pd.DataFrame(yds)
Xds[0,:]

#scaler = MinMaxScaler()
#scaler.fit(Xds)
#Xds = scaler.transform(Xds)

In [3]:
n_inputs = 3
n_hidden1 = 4
n_hidden2 = 2
n_outputs = 1
Xtf = tf.placeholder(tf.float32, shape=(None, n_inputs), name="Xtf")
ytf = tf.placeholder(tf.int32, shape=(None, n_outputs), name="ytf")

In [4]:
## Xavier initialization
#with tf.name_scope("dnn"):
#    hidden1 = tf.layers.dense(Xtf, n_hidden1, name="hidden1", activation=tf.nn.relu)
#    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
#    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

## He initialization
#with tf.name_scope("dnn"):
#    he_init = tf.contrib.layers.variance_scaling_initializer()
#    hidden1 = tf.layers.dense(Xtf, n_hidden1, name="hidden1", activation=tf.nn.relu, kernel_initializer=he_init)
#    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
#    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

## He initialization with batch normalization and dropout

training = tf.placeholder_with_default(False, shape=(), name='traning')
dropout_rate = 0.5 # = = 1 - keep_prob 
X_drop = tf.layers.dropout(Xtf, dropout_rate, training = training)
with tf.name_scope("dnn"):
    he_init = tf.contrib.layers.variance_scaling_initializer()
    hidden1 = tf.layers.dense(X_drop, n_hidden1, name="hidden1", kernel_initializer=he_init)
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    bn1 = tf.layers.batch_normalization(hidden1_drop, training=training, momentum=0.9)
    bn1_act = tf.nn.relu(bn1)
    #bn1_act = tf.nn.tanh(bn1)
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    bn2 = tf.layers.batch_normalization(hidden2_drop, training=training, momentum=0.9)
    bn2_act = tf.nn.relu(bn2)
    #bn2_act = tf.nn.tanh(bn2)
    logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
    logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)

In [5]:
tf.__version__

'1.2.1'

In [6]:
logits

<tf.Tensor 'dnn/batch_normalization_3/batchnorm/add_1:0' shape=(?, 1) dtype=float32>

In [17]:
with tf.name_scope("loss"):
    #xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=ytf, logits=logits)
    xentropy = tf.abs(tf.cast(ytf, tf.float32) - logits) 
    loss=tf.sqrt(tf.reduce_sum(tf.square(xentropy, name="loss")))

In [18]:
learning_rate = 0.01
with tf.name_scope("train"):
    #Simple Gradient Decent or Nestrov Accelerated Gradient optimizer
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(loss)

In [19]:
with tf.name_scope("eval"):
    # Convert logits to label indexes
    #correct_pred = tf.nn.in_top_k(logits, tf.cast(ytf, tf.int32), 1)
    correct_pred = tf.argmax(logits, 1)
    # Define an accuracy metric
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name="accuracy")

In [20]:
init = tf.global_variables_initializer()
asver = tf.train.Saver()

In [21]:
n_epochs = 40
batch_size = 50
num_iters = 10
tf.set_random_seed(1234)
X_train = np.transpose(Xds[:1000,:]).reshape(3, 1000)
X_test = np.transpose(Xds[1001:,:]).reshape(3, 357)
y_train = np.transpose(yds[:1000]).reshape(1, 1000)
y_test = np.transpose(yds[1001:]).reshape(1, 357)
l = len(np.transpose(X_train))
total_batch = int(l / batch_size)
Xtf_batches = np.array_split(X_train, total_batch, axis=1)
ytf_batches = np.array_split(y_train, total_batch, axis=1)
ytf_batches[1].shape

(1, 50)

In [22]:
with tf.Session() as sess:
    init.run()
    #for j in range(num_iters):
    for epoch in range(n_epochs):
        for i in range(total_batch):
            batch_X = Xtf_batches[i]
            batch_y = ytf_batches[i]
            _, loss_value = sess.run([training_op, loss], feed_dict={Xtf: np.transpose(batch_X), ytf: np.transpose(batch_y)})
    predicted = sess.run([correct_pred], feed_dict={Xtf: np.transpose(X_test)})
    Z = logits.eval( feed_dict ={ Xtf: np.transpose(X_test)})
    y_pred = np.argmax(Z, axis = 1) 

In [23]:
print(predicted)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0

In [ ]:
print(y_test)

In [ ]:
print(y_pred)

In [ ]:
ytf